<a href="https://colab.research.google.com/github/topperf15/petrinets/blob/main/sortie_rate_gen_mod_pn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sortie Rate Generation Model Based on Petri Nets, using the pnet.py module developed separately and available from COPADS.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 17 17:34:21 2024

@author: andrew_s
"""

#import sys
#sys.path.append('/Users/andrew_s/Desktop')

import pnet as pn
import random

vul = 6

net = pn.PNet()
# The ingredients to make one sortie
net.add_places('hgr', {'ac': 4})
net.add_places('FL', {'ac': 4})
net.add_places('air', {'ac': 0, 'pilot': 0})
net.add_places('lost', {'ac': 0})
net.add_places('downed', {'pilot': 0})
net.add_places('readyrm', {'pilot': 8})
net.add_places('rest', {'pilot': 2})

# net.add_rules('fly', 'delay', ['FL.ac -> air.ac; 1; 1'])
net.add_rules('land', 'delay', ['air.ac -> hgr.ac; 1; 6'])
net.add_rules('rest', 'delay', ['air.pilot -> rest.pilot; 1; '+str(vul)])
net.add_rules('ready', 'delay', ['hgr.ac -> FL.ac; 1; 3'])
net.add_rules('wait', 'delay', ['rest.pilot -> readyrm.pilot; 1; 8'])

def launch_sortie(places):
    place = places['air']
    ac = place.attributes['ac']
    if ac<4: return 4-ac
    else: return 0

net.add_rules('launch', 'function', ['FL.ac -> air.ac', launch_sortie, 'FL.ac>0; readyrm.pilot>0'])

def launch_pilot(places):
    place = places['air']
    ac = place.attributes['ac']
    pilot = place.attributes['pilot']
    return ac-pilot

net.add_rules('go_fly', 'function', ['readyrm.pilot -> air.pilot', launch_pilot, 'air.ac>0; readyrm.pilot>0'])

def loose_sortie(places):
    place = places['air']
    ac = place.attributes['ac']
    num = random.random()
    Ps = .92
    if num >= Ps and ac>1: return 1
    else: return 0

net.add_rules('lose', 'function', ['air.ac -> lost.ac', loose_sortie, 'air.ac>0'])

def loose_pilot(places):
    lostac = places['lost']
    ac = lostac.attributes['ac']
    downedac = places['downed']
    pilot = downedac.attributes['pilot']
    return ac-pilot

net.add_rules('no_fly', 'function', ['air.pilot -> downed.pilot', loose_pilot, 'air.ac>0; air.pilot>0'])

net.simulate(24, 1, 1)
# Generate results file
data = net.report_tokens()
headers = ['timestep'] + data[0][1]
f = open('flight.csv', 'w')
f.write(','.join(headers) + '\n')
for tdata in data:
    tdata = [tdata[0]] + [str(x) for x in tdata[2]]
    f.write(','.join(tdata) + '\n')
f.close()